# Pathway
> [Pathway](https://pathway.com/) is an open data processing framework. It allows you to easily develop data transformation pipelines and Machine Learning applications that work with live data sources and changing data.

This notebook demonstrates how to set up a live data indexing pipeline. Using `Pathway`, you can query the results of this pipeline from your chains in the same manner as you would a regular vector store. However, under the hood, Pathway updates the index on each data change giving you always up-to-date answers.

In this notebook, we will use a simple document processing pipeline that:

1. Monitors several data sources (files, S3 folders, cloud storages) for data changes.
2. Parses, splits and embeds the documents using `LangChain` methods.
3. Builds a vector index for the data.

We will connect to the index using a `VectorStore` client, which implements the `similarity_search` function to retrieve matching documents.

The basic pipeline described in this document allows to effortlessly build a simple vector index of files stored in a cloud location. However, Pathway provides everything needed to build realtime data pipelines and apps, including SQL-like able operations such as groupby-reductions and joins between disparate data sources, time-based grouping and windowing of data, and a wide array of connectors.


## Prerequisites

Install the `pathway` package. To use the most powerful, `unstructured.io` based parser also install unstructured package.

In [ ]:
# !pip install pathway


## Building the data pipeline

First, make sure you have an API key ith an LLM provider such as OpenAI.

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

 We will now assemble the data vectorization pipeline, using a simple `UTF8` file parser, `CharacterTextSplitter`, and `OpenAIEmbeddings`.

 First, we define the data sources. For simplicity we use the files-based one, but any supported `pathway` [connector](https://pathway.com/developers/api-docs/pathway-io/), such as [s3](https://pathway.com/developers/api-docs/pathway-io-s3/) or [google drive](https://pathway.com/developers/api-docs/pathway-io-gdrive/#pathway.io.gdrive.read) will work too.

 Then, we now define the `LangChain` components that define how the data will be processed.

 Last, we assemble the data pipeline. We will start it running in a background thread, to be able to query it immediately from the demonstration. Please note that in a production deployment, the server will be running in another process, possibly on another machine. For the quick-start, we keep the server and client as different threads of the same python process.

In [ ]:
import pathway as pw
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import PathwayVectorServer

# This creates a `pathway` connector that reads a single file.
data_sources = []
data_sources.append(
    pw.io.fs.read(
        "../../../../templates/rag-pathway/sample_documents",
        format="binary",
        mode="streaming",
        with_metadata=True,
    )
)

# This creates a connector that tracks files in Google drive.
# please follow the instructions at https://pathway.com/developers/tutorials/connectors/gdrive-connector/ to get credentials
# data_sources.append(
#     pw.io.gdrive.read(object_id="17H4YpBOAKQzEJ93xmC2z170l0bP2npMy", service_user_credentials_file="credentials.json", with_metadata=True))

# Choose proper LangChain document transformers
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
embeddings_model = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])

# The PathwayVectorServer is a wrapper over pathway.xpacks.llm.vector_store to accept LangChain transforments
# Fell free to fork it to develop bespoke document processing pipelines.
vector_server = PathwayVectorServer(
    *data_sources,
    embedder=embeddings_model,
    splitter=text_splitter,
)
vector_server.run_server(host="127.0.0.1", port=8765, threaded=True, with_cache=False)

 We now instantiate and configure the client

In [ ]:
from langchain_community.vectorstores import PathwayVectorClient

client = PathwayVectorClient(
    host="127.0.0.1",
    port=8765,
)

 And we can start asking queries

In [ ]:
query = "What is Pathway?"
docs = client.similarity_search(query)

In [ ]:
print(docs[0].page_content)

 **Your turn!** Now make a change to the source documents or make a fresh one and retry the query!

## Filtering based on file metadata

We support document filtering using [jmespath](https://jmespath.org/) expressions, for instance:

In [ ]:
# take into account only sources modified later than unix timestamp
docs = client.similarity_search(query, metadata_filter="modified_at >= `1702672093`")

# take into account only sources modified later than unix timestamp
docs = client.similarity_search(query, metadata_filter="owner == `james`")

# take into account only sources with path containing 'repo_readme'
docs = client.similarity_search(query, metadata_filter="contains(path, 'repo_readme')")

# and of two conditions
docs = client.similarity_search(
    query, metadata_filter="owner == `james` && modified_at >= `1702672093`"
)

# or of two conditions
docs = client.similarity_search(
    query, metadata_filter="owner == `james` || modified_at >= `1702672093`"
)

## Getting information on indexed files

 `PathwayVectorClient.get_vectorstore_statistics()` gives essential statistics on the state of the vector store, like the number of indexed files and the timestamp of last updated one. You can use it in your chains to tell the user how fresh is your knowledge base.

In [ ]:
client.get_vectorstore_statistics()

## Further Configuration

### Parsing Documents

The vectorization pipeline supports pluggable parsers. If not provided, defaults to `UTF-8` parser. You can find available parsers [here](https://github.com/pathwaycom/pathway/blob/main/python/pathway/xpacks/llm/parser.py).

An example parser that can read PDFs, Word documents and other formats is provided with `parsers.ParseUnstructured`: 

In [ ]:
from pathway.xpacks.llm import parsers

parser = parsers.ParseUnstructured()

vector_server = PathwayVectorServer(
    *data_sources,
    embedder=embeddings_model,
    splitter=text_splitter,
    parser=parser,  # we set the parser in the Vector Server
)

### Configuring the cache
The Pathway vectorizing pipeline comes with an embeddings cache:

In [ ]:
vector_server.run_server(..., with_cache=True)

The default cache configuration is the locally hosted disk cache, stored in the `./Cache` directory. However, it is possible to customize it by explicitly specifying the caching backend that can be chosen among several persistent backend [options](https://pathway.com/developers/api-docs/persistence-api/#pathway.persistence.Backend).

### Running in production
A production deployment will typically run the server in a separate process. We provide a template application under [`templates`](https://github.com/langchain-ai/langchain/tree/master/templates/rag-pathway). We recommend to run the Pathway data indexing pipeline in a container-based deployment environment, such as Docker or Kubernetes. For more info see [Pathway's deployment guide](https://pathway.com/developers/user-guide/deployment/).